In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Import cities file as DataFrame
city_weather_df = pd.read_csv("output_data/WeatherDataCities.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kapaa,22.08,-159.32,80.60,78,40,13.87,US,1602588676
1,ancud,-41.87,-73.82,51.01,100,90,10.29,CL,1602588676
2,gazli,40.13,63.45,71.53,20,0,14.61,UZ,1602588676
3,college,64.86,-147.80,26.60,49,90,11.41,US,1602588677
4,cape town,-33.93,18.42,66.99,48,12,18.34,ZA,1602588644
...,...,...,...,...,...,...,...,...,...
585,richards bay,-28.78,32.04,85.06,42,0,8.70,ZA,1602588771
586,sorochinsk,52.43,53.16,62.51,29,74,8.16,RU,1602588771
587,mataura,-46.19,168.86,48.00,75,99,15.01,NZ,1602588678
588,north myrtle beach,33.82,-78.68,71.60,94,90,3.36,US,1602588640


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = city_weather_df[["Lat", "Lng"]]
humidity = city_weather_df["Humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(center=(50.0, 5), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=150,
                                 point_radius=4)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# drop "Date" column 
city_weather_df = city_weather_df[["City","Country","Lat", "Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]]
city_weather_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,kapaa,US,22.08,-159.32,80.60,78,40,13.87
1,ancud,CL,-41.87,-73.82,51.01,100,90,10.29
2,gazli,UZ,40.13,63.45,71.53,20,0,14.61
3,college,US,64.86,-147.80,26.60,49,90,11.41
4,cape town,ZA,-33.93,18.42,66.99,48,12,18.34
...,...,...,...,...,...,...,...,...
585,richards bay,ZA,-28.78,32.04,85.06,42,0,8.70
586,sorochinsk,RU,52.43,53.16,62.51,29,74,8.16
587,mataura,NZ,-46.19,168.86,48.00,75,99,15.01
588,north myrtle beach,US,33.82,-78.68,71.60,94,90,3.36


In [7]:
# Customize DataFrame by requested conditions.
ideal_weather_df = city_weather_df.loc[(city_weather_df["Max Temp"]>=70) & (city_weather_df["Max Temp"]<=80) &
                   (city_weather_df["Wind Speed"]<=10) & (city_weather_df["Cloudiness"]==0)].dropna()
ideal_weather_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
69,castro,BR,-24.79,-50.01,75.96,43,0,1.12
177,puerto del rosario,ES,28.50,-13.86,77.00,73,0,8.05
218,castro,BR,-24.79,-50.01,75.96,43,0,1.12
272,garmsar,IR,35.22,52.34,73.63,18,0,3.02
305,castro,BR,-24.79,-50.01,75.96,43,0,1.12
399,castro,BR,-24.79,-50.01,75.96,43,0,1.12
463,castro,BR,-24.79,-50.01,75.96,43,0,1.12
490,tezu,IN,27.92,96.17,79.97,66,0,3.02


In [8]:
# Store into variable named hotel_df
hotel_df = ideal_weather_df[["City","Country","Lat","Lng"]]

# Add a new column as "Hotel Name" 
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-8-bd34b836b028>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Country,Lat,Lng,Hotel Name
69,castro,BR,-24.79,-50.01,
177,puerto del rosario,ES,28.50,-13.86,
218,castro,BR,-24.79,-50.01,
272,garmsar,IR,35.22,52.34,
305,castro,BR,-24.79,-50.01,
399,castro,BR,-24.79,-50.01,
463,castro,BR,-24.79,-50.01,
490,tezu,IN,27.92,96.17,


In [9]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}
# Loop through the hotel_df 
for index, row in hotel_df.iterrows():
    
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    response = requests.get(base_url, params=params)
    
    # convert to json
    response = response.json()
    results = response['results']
    try:
        print(f"Nearest Hotel in {hotel_df['City']} is {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"] 
    except IndexError:
        print("Missing result ... skipping.")
    print("------------")

Nearest Hotel in 69                 castro
177    puerto del rosario
218                castro
272               garmsar
305                castro
399                castro
463                castro
490                  tezu
Name: City, dtype: object is Urano Business Hotel
------------


/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Nearest Hotel in 69                 castro
177    puerto del rosario
218                castro
272               garmsar
305                castro
399                castro
463                castro
490                  tezu
Name: City, dtype: object is Hotel El Mirador
------------
Nearest Hotel in 69                 castro
177    puerto del rosario
218                castro
272               garmsar
305                castro
399                castro
463                castro
490                  tezu
Name: City, dtype: object is Urano Business Hotel
------------
Nearest Hotel in 69                 castro
177    puerto del rosario
218                castro
272               garmsar
305                castro
399                castro
463                castro
490                  tezu
Name: City, dtype: object is هتل شهر آفتاب Sun City
------------
Nearest Hotel in 69                 castro
177    puerto del rosario
218                castro
272               garmsar
305              

In [10]:
# Display DataFrame
hotel_df

,City,Country,Lat,Lng,Hotel Name
69,castro,BR,-24.79,-50.01,Urano Business Hotel
177,puerto del rosario,ES,28.50,-13.86,Hotel El Mirador
218,castro,BR,-24.79,-50.01,Urano Business Hotel
272,garmsar,IR,35.22,52.34,هتل شهر آفتاب Sun City
305,castro,BR,-24.79,-50.01,Urano Business Hotel
399,castro,BR,-24.79,-50.01,Urano Business Hotel
463,castro,BR,-24.79,-50.01,Urano Business Hotel
490,tezu,IN,27.92,96.17,Capital O 66991 Hotel Highway


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))